In [63]:
import numpy as np
import openturns as ot

## Test conditionnement avec OT mais comportement similaire à Vraisemblance

In [64]:
#     conditionedDist = ot.Normal()

#     mu = np.zeros(2)
#     cov = ot.CovarianceMatrix(np.array([[1.0 , 1.8], [1.8 , 4.0]]))
#     conditioningDist = ot.Normal(mu,cov)


#     finalDist = ot.ConditionalDistribution(conditionedDist, conditioningDist)
#     #finalDist.getSample(3)

## Modèle gaussien avec 3 entrées

#### Définition du modèle

In [65]:
def model(X):
    return np.sum(X,1)

moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

#dependent_ind = np.array([0,1])
#given_ind = np.array([2])

#### Calculs moyenne et variance conditionnelle pour vecteur gaussien

###### Formule poly Roman

In [66]:
def condMVN(mean, cov, dependent_ind, given_ind, X_given):
    """ Returns conditional mean and variance of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cov = np.array(cov)
    
    B = cov.take(dependent_ind, axis = 1)
    B = B[dependent_ind]
    
    C = cov.take(dependent_ind, axis = 1)
    C = C[given_ind]
    
    D = cov.take(given_ind, axis = 1)
    D = D[given_ind]
    
    CDinv = np.dot(np.transpose(C),np.linalg.inv(D))
    
    condMean = mean[dependent_ind] + np.dot(CDinv,(X_given - mean[given_ind]))
    condVar = B - np.dot(CDinv,C)
    condVar = ot.CovarianceMatrix(condVar)
    
    return condMean,condVar

#condMVN(moyenne, cov, dependent_ind = np.array([1,0]), given_ind= np.array([2]), X_given= np.array([10]))

###### Formule package condMVNorm

In [67]:
# def condMVN(mean, cov, dependent_ind, given_ind, X_given):
#     """ Returns conditional mean and variance of X[dependent.ind] | X[given.ind] = X.given
#     where X is multivariateNormal(mean = mean, covariance = cov)"""
    
#     cov = np.array(cov)
    
#     B = cov.take(dependent_ind, axis = 1)
#     B = B[dependent_ind]
    
#     C = cov.take(given_ind, axis = 1)
#     C = C[dependent_ind]
    
#     D = cov.take(given_ind, axis = 1)
#     D = D[given_ind]
    
#     CDinv = np.dot(C,np.linalg.inv(D))
    
#     condMean = mean[dependent_ind] + np.dot(CDinv,(X_given - mean[given_ind]))
#     condVar = B - np.dot(CDinv,np.transpose(C))
#     condVar = ot.CovarianceMatrix(condVar)
    
#     return condMean,condVar

# condMVN(moyenne, cov, dependent_ind = np.array([1,0]), given_ind= np.array([2]), X_given= np.array([10]))

#### Génération loi conditionnelle

In [68]:
def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cond_mean,cond_var = condMVN(mean, cov, dependent_ind, given_ind, X_given)
    distribution = ot.Normal(cond_mean,cond_var)
    return distribution.getSample(n)
    
#r_condMVN(5,moyenne, cov, dependent_ind, given_ind, X_given)

## Définition des indices de Shapley

In [69]:
Nv = 10**4
No = 10**3
Ni = 3

d = 3
perms_tool = ot.KPermutations(d, d)
perms = perms_tool.generate()

m = perms.getSize() # nombre de permutation

def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

#### Conception de la matrice de design

In [71]:
X = np.zeros((Nv+m*(d-1)*No*Ni, d)) 
X[:Nv,:] = Xall(Nv)

for p in range(m):
    
    pi = perms[p]
    pi_sorted = np.argsort(pi)
    
    for j in range(1,d):
        
        Sj = pi[:j] # set of the 1st-jth elements in pi      
        Sjc = pi[j:] # set of the (j+1)th-dth elements in pi
        
        xjcM = Xcond(No,Sjc,None,None)# sampled values of the inputs in Sjc

        for l in range(No):

            xjc = xjcM[l,:]
            
            # sample values of inputs in Sj conditional on xjc
            xj = Xcond(Ni, Sj, Sjc, xjc)
            xx = np.concatenate((xj, np.ones((Ni,1))*xjc), axis = 1)
            ind_inner = Nv + p*(d-1)*No*Ni + (j-1)*No*Ni + l*Ni
            X[ind_inner:(ind_inner + Ni),:] = xx[:,pi_sorted]

In [83]:
# Calcul de la réponse
y = model(X)

# Initialize Shapley value for all players
Sh = np.zeros(d)
  
# Initialize main and total (Sobol) effects for all players
Vsob = np.zeros(d)
Tsob = np.zeros(d)
  
# Estimate Var[Y] 
Y = y[:Nv]
y = y[Nv:]
EY = np.mean(Y)
VarY = np.var(Y)

In [84]:
# Estimate Shapley effects
cVar = np.zeros(No)
count = 0

for p in range(m):
    
    pi = perms[p]
    prevC = 0
    
    for j in range(d):
        if (j == (d-1)):
            Chat = VarY
            delta = Chat - prevC
            Vsob[pi[j]] = Vsob[pi[j]] + prevC # first order effect
        else:
            for l in range(No):
                Y = y[:Ni]
                count = count + len(Y)
                y = y[Ni:]
                cVar[l] = np.var(Y)
            Chat = np.mean(cVar)
            delta = Chat - prevC
      
        Sh[pi[j]] = Sh[pi[j]] + delta
        
        prevC = Chat
        
        if (j == 0):
            Tsob[pi[j]] = Tsob[pi[j]] + Chat # Total effect

In [87]:
Sh = Sh / m / VarY

Vsob = Vsob / (m/d) / VarY # averaging by number of permutations with j=d-1
Vsob = 1 - Vsob 

Tsob = Tsob / (m/d) / VarY # averaging by number of permutations with j=1 

print(Sh)
print(Vsob)
print(Tsob)

[ 0.00306428  0.00683989  0.00758187]
[ 0.97986012  0.95404604  0.95193617]
[ 0.00378352  0.00069101  0.00281548]


In [122]:
np.array([1,2,2],dtype=np.int64)

array([1, 2, 2], dtype=int64)